In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html


import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from CRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################





username = "Myadmin"
password = "Admin"
database = 'admin'
shelter = AnimalShelter(username, password, database)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


app.layout = html.Div([

    html.Center(html.B(html.H1('Garrett Waag SNHU Admin Dashboard'))),
    html.Hr(),
   
    html.Div([
    html.Label('Name:'),
    dcc.Input(id='name'),
    html.Br(),
    html.Label('Age Upon Outcome:'),
    dcc.Input(id='age_upon_outcome'),
    html.Br(),
    html.Label('Breed:'),
    dcc.Input(id='breed'),
    html.Br(),
    html.Label('Animal Id:'),
    dcc.Input(id='animal_id'),
    html.Br(),
    html.Label('Animal Type:'),
    dcc.Input(id='animal_type'),
    html.Br(),
    html.Label('Color:'),
    dcc.Input(id='color'),
    html.Br(),
    html.Label('Date Of Birth:'),
    dcc.Input(id='date_of_birth'),
    html.Br(),
    html.Label('Date Time:'),
    dcc.Input(id='date_time'),
    html.Br(),
    html.Label('Month Year:'),
    dcc.Input(id='month_year'),    
    html.Br(),
    html.Label('Outcome Subtype:'),
    dcc.Input(id='outcome_subtype'),
    html.Br(),
    html.Label('Outcome Type:'),
    dcc.Input(id='outcome_type'),
    html.Br(),
    html.Label('Sex Upon Outcome:'),
    dcc.Input(id='sex_upon_outcome'),
    html.Br(),
    html.Label('Location Latitude:'),
    dcc.Input(id='location_lat'),
    html.Br(),
    html.Label('Location Longitude:'),
    dcc.Input(id='location_long'),
    html.Br(),
    html.Label('Age Upon Outcome In Weeks:'),
    dcc.Input(id='age_upon_outcome_in_weeks'),      
    html.Br(),
    
    ]),
    
    
    html.Button(id='create', n_clicks=0, children='Create'),
    html.Button(id='update', n_clicks=0, children="Update Selected"),
    html.Button(id='delete', n_clicks=0, children="Delete Seleced"),
    html.Br(),
    html.Div(id='create-status'),
    html.Div(id='update-status'),
    #html.Div(id='delete-status'),
    
    
    
    
    
    
    
    
    html.Hr(),
    
    #data filter buttons for datatable
    html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    dcc.RadioItems(id='radio-items',
                        options=[
                        {'label': 'Water', 'value': 'Water'},
                        {'label': 'Mountain/Wilderness', 'value': 'Wild'},
                        {'label': 'Disaster/Individual', 'value': 'Indi'},
                        {'label': 'Reset', 'value': 'Reset'}
                        ],
                        value='Reset'
                    ),
                ]),
    html.Hr(),
    
    
    #sets datatable to only select 1 row and can sort by any column
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_current= 0,
        page_size= 10, 
        row_selectable='single',
        selected_columns=[],
        selected_rows=[0],
        
    ),
    html.Br(),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-items', 'value')])

#sets filters for db
def update_dashboard(value):
    
    if (value == 'Reset'):
        df = pd.DataFrame(shelter.read({}))
    
    elif (value == 'Water'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['Labrador Retriever Mix','Chesapeake Bay Retriever','Newfoundland']},
            'sex_upon_outcome':'Intact Female','age_upon_outcome_in_weeks':{'$gte':26},
             'age_upon_outcome_in_weeks':{'$lte':156}}))
    elif(value == 'Wild'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['German Shepherd','Alaskan Malamute','Old English Sheepdog','Siberian Husky','Rottweiler']},
            'sex_upon_outcome':'Intact Male','age_upon_outcome_in_weeks':{'$gte':26},
             'age_upon_outcome_in_weeks':{'$lte':156}}))
        
    elif(value == 'Indi'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['German Shepherd','Doberman Pinscher','Golden Retriever','Bloodhound','Rottweiler']},
            'sex_upon_outcome':'Intact Male','age_upon_outcome_in_weeks':{'$gte':20},
             'age_upon_outcome_in_weeks':{'$lte':300}}))
    
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(Output('datatable-id', 'style_data_conditional'),
            [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback(Output('create-status', 'children'),
             [Input('create', 'n_clicks'),
              State('name', 'value'),
              State('age_upon_outcome', 'value'),
              State('breed', 'value'),
              State('animal_id', 'value'),
              State('animal_type', 'value'),
              State('color', 'value'),
              State('date_of_birth', 'value'),
              State('date_time', 'value'),
              State('month_year', 'value'),
              State('outcome_subtype', 'value'),
              State('outcome_type', 'value'),
              State('sex_upon_outcome', 'value'),
              State('location_lat', 'value'),
              State('location_long', 'value'),
              State('age_upon_outcome_in_weeks', 'value')])
def create_entry(n_clicks, name, age_upon_outcome, breed, animal_id, animal_type, color, date_of_birth,
                date_time, month_year, outcome_subtype, outcome_type, sex_upon_outcome, location_lat, location_long,
                age_upon_outcome_in_weeks):
    shelter.create({'name': name,'age_upon_outcome':age_upon_outcome, 'breed': breed,'animal_id': animal_id,'animal_type':animal_type,
                   'color': color,'date_of_birth':date_of_birth,'date_time': date_time,'month_year':month_year,
                   'outcome_subtype': outcome_subtype,'outcome_type':outcome_type,'sex_upon_outcome': sex_upon_outcome,
                    'location_lat':location_lat, 'location_long': location_long, 'age_upon_outcome_in_weeks': age_upon_outcome_in_weeks})
    if n_clicks > 0:
        return u'''Created entry'''.format()
    
   


@app.callback(Output('update-status', 'children'),
             [Input('update', 'n_clicks'),
              State('datatable-id', "derived_viewport_data"),
              State('datatable-id', "selected_rows"),
              State('name', 'value'),
              State('age_upon_outcome', 'value'),
              State('breed', 'value'),
              State('animal_id', 'value'),
              State('animal_type', 'value'),
              State('color', 'value'),
              State('date_of_birth', 'value'),
              State('date_time', 'value'),
              State('month_year', 'value'),
              State('outcome_subtype', 'value'),
              State('outcome_type', 'value'),
              State('sex_upon_outcome', 'value'),
              State('location_lat', 'value'),
              State('location_long', 'value'),
              State('age_upon_outcome_in_weeks', 'value')])
def update_entry(n_clicks,viewData, selected_row, name, age_upon_outcome, breed, animal_id, animal_type, color, date_of_birth,
                date_time, month_year, outcome_subtype, outcome_type, sex_upon_outcome, location_lat, location_long,
                age_upon_outcome_in_weeks):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    target = [dff.to_dict('records'), selected_row]
    data = {'name': name,'age_upon_outcome':age_upon_outcome, 'breed': breed,'animal_id': animal_id,'animal_type':animal_type,
        'color': color,'date_of_birth':date_of_birth,'date_time': date_time,'month_year':month_year,
        'outcome_subtype': outcome_subtype,'outcome_type':outcome_type,'sex_upon_outcome': sex_upon_outcome,
        'location_lat':location_lat, 'location_long': location_long, 'age_upon_outcome_in_weeks': age_upon_outcome_in_weeks}
    #shelter.update({'animal_id':target},
     #   )
    if n_clicks > 0:
        return target
 



# @app.callback(Output('create-status', "children"),
   # [Input('delete', 'n_clicks'),
   # Input('datatable-id', "derived_viewport_data"),
  #  Input('datatable-id', "selected_rows")])  
#def delete_entry(n_clicks, viewData, row):

#    animal_id = dff.loc[row[0], 'animal_id']
  #  shelter.delete({'animal_id' : animal_id})
    
   # if n_clicks > 0:
   #     return u'''deleted entry'''

app